# PV_Analysis Ophiucus 1


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvanalysis import PVAnalysis

## C18O

### Mean method

In [ ]:

'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C18O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C18O_mTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 2.56e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-300, 0, 0, 300] # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-3.5, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'


#Caso1
#Cuando dp es falso y pin es falso, el disco se va a mover keplerianamente con pin=0.5 y lo que obtengo del fit es el pout.
#no funciono 

#Siempre tomar dp =TRUE

#Caso2
#dp= TRUE, pin=TRUE, encuentra pin y pout para los datos.

#Testear con outlimit los casos en que hay absorción.

#Los puntos .dat no cambian, se obtienen los mismos



#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 163.77
deltarb= 60.11
vb= 2.291
pin=0.522
dp=0.236

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,5)
plt.xlim(20,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1 : C18O  ')
plt.suptitle("Mean method", fontsize='15')
plt.savefig('GRAFICO' +outname, dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,5)
plt.xlim(20,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1 : C18O  ')
plt.suptitle("Mean method", fontsize='15')
plt.savefig('GRAFICO' +outname + 'abs', dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Gaussian method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C18O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C18O_gTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 2.56e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'gauss'  # 'mean' or 'gauss'
xlim = [-300, 0, 0, 300]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-3.5, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
##PV_ANALYSIS

impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=True, plotedgepoint=True,
                    plotridgemodel=True, plotedgemodel=True)

#### Double Power_Law

In [ ]:
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 164.72
deltarb= 62.13
vb=2.284
pin=0.524
dp=0.227

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,5)
plt.xlim(minimo_grafico,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C18O')
plt.suptitle("gauss method , RMS=2.56", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.show()
plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,5)
plt.xlim(minimo_grafico,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C18O')
plt.suptitle("gauss method , RMS=2.56", fontsize='15')
plt.savefig("GRAFICO"+ outname+'ABS', dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Velocidad acotada/ Mean method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C18O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C18O_mTTT_tr3_vlim'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 2.56e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-300, 0, 0, 300]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-3.5, -1, 1, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=True, plotedgepoint=True,
                    plotridgemodel=True, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)



#Parametros
rb= 160.68
deltarb= 55.62
vb=2.323
pin=0.520
dp=0.223

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,5)
plt.xlim(minimo_grafico,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C18O')
plt.suptitle("Vel acotada (mean)", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,5)
plt.xlim(minimo_grafico,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C18O')
plt.suptitle("Vel acotada (mean)", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

## C17O

### Mean method

In [ ]:

'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C17O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C17O_mTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 1.22e-3  # Jy/beam
thr = 3.5  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-250, 0, 0, 250]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-4, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#rms del orden de 1e-3

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:

#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)




#Parametros
rb= 158.84
deltarb= 8.66
vb=2.598
pin=0.614
dp=0.470

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Mean method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Mean method, thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Gauss method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C17O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C17O_gTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 1.22e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'gauss'  # 'mean' or 'gauss'
xlim = [-250, 0, 0, 250]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-4, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=True, plotedgepoint=True,
                    plotridgemodel=True, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)



#Parametros
rb= 159.63
deltarb=  9.31
vb=2.669
pin= 0.475
dp= 0.646


pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,6)
plt.xlim(10,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 85,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,6)
plt.xlim(10,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 85,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Thr=4 Mean method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C17O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C17O_mTTT_tr4'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 1.22e-3  # Jy/beam
thr = 4.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-250, 0, 0, 250]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-4, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=True, plotedgepoint=True,
                    plotridgemodel=True, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 158.43
deltarb=  63.35
vb=2.415
pin= 0.688
dp=0.210


pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)-20

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,6)
plt.xlim(10,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 85,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=4", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,6)
plt.xlim(10,max_grafico)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 85,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=4", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Thr=3 Mean method, pin=0.5

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_C17O_pvdiagram.fits'
outname = 'pvanalysis_Oph1_C17O_mTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 1.22e-3  # Jy/beam
thr = 3  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-250, 0, 0, 250]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-4, 0, 0, 8]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = False  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)

#Parametros
rb= 161.76
deltarb= 8.76
vb=2.623
pin=0.5
dp=0.59

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'pin=05' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: C17O')
plt.suptitle("Gauss method,thr=3,pin=0.5", fontsize='15')
plt.savefig("GRAFICO"+ outname+'pin=05 abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

## 12CO

### Mean method/ Velocidad acotada

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_12CO_pvdiagram.fits'
outname = 'pvanalysis_Oph1_12CO_mTTT_tr3_vlim_1'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 3.42e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-380, 0, 0,0]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-6,0,0,0]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'


#test 1 
#Pin=kepler
#usar un corte (probar con ambos


#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')

name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 110.58
deltarb= 140.42
vb=3.5
pin=0.59
dp=0.175

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])


max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("mean method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("mean method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Gauss method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_12CO_pvdiagram.fits'
outname = 'pvanalysis_Oph1_12CO_gTTT_tr3_vlim'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 3.42e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'gauss'  # 'mean' or 'gauss'
xlim = [-380, 0, 0, 380]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-6, 0,0,0]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law


In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 257.34
deltarb= 151.26
vb=1.789
pin=0.567
dp=0.281

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])


max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### Treshure=4

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_12CO_pvdiagram.fits'
outname = 'pvanalysis_Oph1_12CO_gTTT_tr4_vlim'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 3.42e-3  # Jy/beam
thr = 4.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-380, 0, 0, 50]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-6, 0, 0, 0]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)

#Parametros
rb= 129.88
deltarb= 143.64
vb=2.062
pin=0.699
dp=0.079

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])


max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')

plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

### pin=0.5 thr=3 /mean method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_12CO_pvdiagram.fits'
outname = 'pvanalysis_Oph1_12CO_mTTT_tr3_pin=05'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 3.42e-3  # Jy/beam
thr = 3.  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-380, 0, 0, 380]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-6, 0, 0, 0]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = False  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)

#Parametros
rb= 200.30
deltarb= 101.96
vb=2.511
pin=0.442
dp=0.198

pout=pin+dp
x=np.arange(0,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])
minimo_grafico=min(positivos)

max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'pin=05' , dpi=300, bbox_inches='tight')

plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Gauss method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'pin=05 abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()

## 13CO

### Mean method

In [ ]:
'-------- INPUTS --------'
fitsfile = './Ophiucus 1/Oph1_13CO_pvdiagram.fits'
outname = 'pvanalysis_Oph1_13CO_mTTT_tr3'  # file name header for outputs
incl = 71.  # deg
vsys = 2.3# km/s
dist = 139.  # pc
rms = 3.65e-3  # Jy/beam
thr = 5  # rms
ridgemode = 'mean'  # 'mean' or 'gauss'
xlim = [-450, 0, 0, 210]  # au; [-outlim, -inlim, inlim, outlim]
vlim = np.array([-5, 0, 0, 0]) + vsys  # km/s
Mlim = [0, 10]  # M_sun; to exclude unreasonable points
xlim_plot = [375./20. , 375.]  # au; [inlim, outlim]
vlim_plot = [6./20. , 13.]  # km/s
use_velocity = True  # cuts along the velocity direction
use_position = True  # cuts along the positional direction
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = True  # False means a single power
include_pin = True  # False means pin=0.5 (Keplerian).
show_pv = True  # figures will be made regardless of this option.
show_corner = True  # figures will be made regardless of this option.
minabserr = 0.1  # minimum absolute errorbar in the unit of bmaj or dv.
minrelerr = 0.01  # minimum relative errorbar.
R_dust=45.66
'------------------------'

#### PV_analysis

In [ ]:
impv = PVAnalysis(fitsfile, rms, vsys, dist, pa=51.34)
impv.get_edgeridge(outname, thr=thr, ridgemode=ridgemode, incl=incl,
                   use_position=use_position, use_velocity=use_velocity,
                   Mlim=Mlim, xlim=np.array(xlim) / dist, vlim=vlim,
                   minabserr=minabserr, minrelerr=minrelerr,
                   nanbeforemax=True, nanopposite=True, nanbeforecross=True)
impv.write_edgeridge(outname=outname)
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()
impv.plot_fitresult(vlim=vlim_plot, xlim=xlim_plot, flipaxis=False,
                    clevels=[-9,-6,-3,3,6,9], outname=outname,
                    show=show_pv, logcolor=True, Tbcolor=False,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'},
                    plotridgepoint=False, plotedgepoint=True,
                    plotridgemodel=False, plotedgemodel=True)

#### Double Power-Law

In [ ]:
#file= open("pvanalysis.edge.dat",'r')
name=outname+'.edge.dat'
file=open(name,'r')
texto=file.read()
lista= texto.split()
np.shape(lista)


#Parametros
rb= 245.35
deltarb= 97.48
vb=1.489
vb=1.8
pin=0.602
dp=1.411

pout=pin+dp
x=np.arange(0.1,1000,0.1)



for i in range(9):
    print(lista[0])
    lista.pop(0)
    
    
len(lista)
#Separamos las 4 filas

fila1=[]
fila2=[]
fila3=[]
fila4=[]

for i in range(len(lista)):
    elemento=float(lista[i])
    if i%4==0:
        fila1.append(elemento)
    
    if i%4==1:
        fila2.append(elemento)
    
    if i%4==2:
        fila3.append(elemento)
        
    if i%4==3:
        fila4.append(elemento)    

Fila1=np.array(fila1)
Fila2=np.array(fila2)
Fila3=np.array(fila3)
Fila4=np.array(fila4)
radios=np.ones(len(fila1))*rb
deltaplus=np.ones(len(fila1))*(rb+deltarb)
deltaminus=np.ones(len(fila1))*(rb-deltarb)
plt.loglog(Fila1,Fila3,'bo')
#plt.plot(radios,fila3, 'ro',markersize=2)
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)

def g(x,p,q):
    #print(x)
    ##print(x[0])
    #print(type(x[0]))
    vel=np.zeros(len(x))
    for i in range(len(x)):
        #print(x[i])
        
        if abs(x[i])<rb:
            vel[i]=vb*(x[i]/rb)**-p
        if abs(x[i])>rb:
            vel[i]=vb*(x[i]/rb)**-q
    
    return vel




#fit=g(Fila1,pin,pout)
fitx=g(x,pin,pout)

pin_round= round(pin,2)
pout_round= round(pout,2)

positivos=[]
#Sacamos el minimo del grafico xlim
for i in range(len(Fila1)):
    if Fila1[i]>0:
        positivos.append(Fila1[i])


max_grafico=max(Fila1)+ 100

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(Fila1,Fila3,'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(Fila1,Fila3,yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Mean method,thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Probamos con valor absoluto

plt.plot(x,fitx,'r')
#plt.loglog(Fila1,fit,'r')
plt.loglog(abs(Fila1),abs(Fila3),'bo')
plt.ylim(1,10)
plt.xlim(5,1000)

ejex=np.arange(0,1,1)
a=plt.axvline(x=rb, ymin=0, ymax=10,color = "black", linewidth = 2, linestyle = "dashed")
b=plt.axvline(x=rb-deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
c=plt.axvline(x=rb+deltarb, ymin=0, ymax=10,color = "gray", linewidth = 1, linestyle = "dashed")
d=plt.axvline(x=R_dust, ymin=0, ymax=10,color = "magenta", linewidth = 2, linestyle = "dashed")

deltaminus=rb-deltarb
deltaplus=rb+deltarb


plt.fill_betweenx(fitx, deltaminus, deltaplus,facecolor='lightgray')
plt.legend([a,d],['$R_{break}$','$R_{dust}$'],loc='upper right',fontsize=15)
              #ax.legend([line1, line2, line3], ['label1', 'label2', 'label3'])

#plt.plot(radios,fila3,'.' )
#plt.plot(deltaplus,fila3, 'mo',markersize=1)
#plt.plot(deltaminus,fila3, 'mo',markersize=1)
plt.text( 50,2.8,'$P_{in}$='+ str(pin_round), fontsize=15, color='black')
plt.text( 100,1.3,'$P_{out}$='+ str(pout_round), fontsize=15, color='black')
plt.errorbar(abs(Fila1),abs(Fila3),yerr=Fila4,xerr=Fila2,fmt = 'bo', ms=1)
plt.title('Ophiucus 1: 12CO')
plt.suptitle("Mean method, thr=3", fontsize='15')
plt.savefig("GRAFICO"+ outname+'abs' , dpi=300, bbox_inches='tight')
plt.show()

########################################################################################################
########################################################################################################
#Mostramos los puntos

plt.plot(Fila1,Fila3,'bo')
plt.title('Puntos representativos')
plt.show()